In [181]:
# !pip install pyomo
# !pip install idaes-pse

In [182]:

import numpy as np
import pandas as pd
# from contextlib import redirect_stdout

from pyomo.environ import *

solvername = 'ipopt'


Caso não encontre o solver na pasta:
- Importar arquivos de extensões da biblioteca IDAES;
- Copiar o arquivo do solver ipopt para a pasta do projeto e excluir os outros arquivos.

In [183]:
# search for solver executable in the PATH

import os

if os.path.exists(solvername + '.exe'):
    print(solvername + ' executable found')

else:
    print(solvername + ' executable not found')

    # ------------------------- # 

    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

    # ------------------------- # 

    import shutil

    # file names
    src_file = "bin\\" + solvername + ".exe"
    dst_file = solvername + ".exe"

    shutil.copyfile(src_file, dst_file)
    print('Copied')

    # ------------------------- #

    # excluir pasta bin
    import os, shutil

    pasta = 'bin'
    shutil.rmtree(pasta)
    print('Deleted')

    # ------------------------- #


ipopt executable found


In [184]:

sig = 0

UTEs = [('UTE1', 'UTE2', 'UTE3', 'UTE4'),
        (1000, 970, 700, 680),
        (16.19, 17.26, 16.6, 16.5),
        (0.00048, 0.00031, 0.002, 0.00211),
        (455, 455, 130, 130),
        (150, 150, 20, 20)]
UTEs = np.transpose(UTEs)
UTE = pd.DataFrame(UTEs, columns=['Nome','a','b','c','PGmax','PGmin'])



Dem = [('Hora1', 450, 45),
       ('Hora2', 530, 53),
       ('Hora3', 600, 60),
       ('Hora4', 540, 54),
       ('Hora5', 400, 40),
       ('Hora6', 280, 28),
       ('Hora7', 290, 29),
       ('Hora8', 500, 50)]
Dem = pd.DataFrame(Dem, columns=['Hora','Dem', 'Res'])

UTE['a'] = UTE['a'].astype('float')
UTE['b'] = UTE['b'].astype('float')
UTE['c'] = UTE['c'].astype('float')
UTE['PGmax'] = UTE['PGmax'].astype('float')
UTE['PGmin'] = UTE['PGmin'].astype('float')


In [185]:

def Otimiza_UC(UTE,Dem,Sig):
    #=====================================Modelagem do Problema============================================================
    modelo = ConcreteModel('Otimizacao Unnit Commitment')

    def limite_UTE(modelo,pg,ext):
        return (UTE[UTE['Nome'] == pg]['PGmin'].values[0], UTE[UTE['Nome'] == pg]['PGmax'].values[0])
    def limite_U(modelo,pg,ext):
        return (0, 1)
    def limite_X(modelo,pg,ext):
        return (0.0, 20.0)

    # ----------------    Declarar Variáveis
    modelo.Pg = Var(UTE['Nome'], Dem['Hora'], bounds=limite_UTE, domain=NonNegativeReals)
    modelo.U = Var(UTE['Nome'], Dem['Hora'], bounds=limite_U, domain=NonNegativeReals) # Função Decisão de Operação - FDO

    if Sig == 1:
        modelo.X = Var(UTE['Nome'], Dem['Hora'], bounds=limite_X, domain=NonNegativeReals) # Argumento da FDO  -  utilizado na aproximação sigmoidal


    # ----------------    Declarar Função Objetivo
    def objective_rule(modelo):
        fob = 0
        for h in Dem['Hora']:
            fob += sum((UTE[UTE['Nome'] == g]['a'].values[0] +
                        UTE[UTE['Nome'] == g]['b'].values[0]*modelo.Pg[g,h] +
                        UTE[UTE['Nome'] == g]['c'].values[0]*modelo.Pg[g,h]**2)*modelo.U[g,h] for g in UTE['Nome'])
        return fob


    modelo.objetivo = Objective(rule=objective_rule,sense=minimize)

    # ----------------     Declarar Restrições
    modelo.AtendDem = ConstraintList()
    modelo.AtendDemRes = ConstraintList()
    modelo.LimGer = ConstraintList()
    # modelo.Sigmoide = ConstraintList()


    for h in Dem['Hora']:
        modelo.AtendDem.add(sum(modelo.U[g,h]*modelo.Pg[g,h] for g in UTE['Nome']) == Dem[Dem['Hora'] == h]['Dem'].values[0])
    for h in Dem['Hora']:
        modelo.AtendDemRes.add(sum(modelo.U[g,h]*UTE[UTE['Nome'] == g]['PGmax'].values[0] for g in UTE['Nome']) >=
                               Dem[Dem['Hora'] == h]['Dem'].values[0] + Dem[Dem['Hora'] == h]['Res'].values[0])
    # for h in Dem['Hora']:
    #     for g in UTE['Nome']:
    #         modelo.LimGer.add(UTE[UTE['Nome'] == g]['PGmin'].values[0] <= modelo.Pg[g,h])
    #         modelo.LimGer.add(modelo.Pg[g,h] <= UTE[UTE['Nome'] == g]['PGmax'].values[0])



    # FDO Sigmoidal - definição

    if Sig == 1:
        modelo.FDO_Sigmoide = ConstraintList() # Restrição da FDO Sigmoidal
        alpha=2
        for h in Dem['Hora']:
            for g in UTE['Nome']:
                modelo.FDO_Sigmoide.add(modelo.U[g,h]  -  (exp(alpha * (modelo.X[g,h])) - 1) / (exp(alpha * (modelo.X[g,h])) + 1) == 0)


    # caso Sig == 0, o algoritmo decidirá o valor de U entre 0 e 1; logo, será feita uma aproximação linear




    #------------------------- Adição da aproximação pela função sigmoide
    # for h in Dem['Hora']:
    #     for g in UTE['Nome']:
    #         modelo.Sigmoide.add(0 <= 1/(1+ exp(-modelo.U[g,h])))
    #         modelo.Sigmoide.add(1/(1+ exp(-modelo.U[g,h])) <= 1)


    # if sig == 0:
    #     modelo.Sigmoide.deactivate()



    # with open('resumo_modelo.txt', 'w') as f:
    #     # Redirecione a saída para o arquivo
    #     with redirect_stdout(f):
    #         # Imprima um resumo do modelo
    #         modelo.pprint()

    # modelo.pprint()



    ### Executar a Otimização.
    # solver = SolverFactory('ipopt')
    # solver = SolverFactory('ipopt',executable='bin/ipopt')
    solver = SolverFactory('ipopt',executable='ipopt.exe')

    resultados_seq = solver.solve(modelo, tee=False)


    # resultados_seq = solver.solve(model_seq, tee=False)
    # resultados_con = solver.solve(model_con, tee=False)

    # Relatório dos resultados de otimização

    print('Status Final do Problema de Otimização:', resultados_seq.solver.status)
    print('Condição de Término:', resultados_seq.solver.termination_condition)
    print('Resultado Função Objetivo:', value(modelo.objetivo), '\n')
    return modelo


In [186]:

modelo_arred = Otimiza_UC(UTE, Dem, 0)

print(1*'\n ' + 50*'-' + 2*'\n')

modelo_sig = Otimiza_UC(UTE, Dem, 1)


Status Final do Problema de Otimização: ok
Condição de Término: optimal
Resultado Função Objetivo: 67863.08835063892 


 --------------------------------------------------


Status Final do Problema de Otimização: ok
Condição de Término: optimal
Resultado Função Objetivo: 67863.08878362423 



In [187]:

Sigm = modelo_sig.Pg.extract_values()
U_sig = modelo_sig.U.extract_values()
X_sig = modelo_sig.X.extract_values()

Arred = modelo_arred.Pg.extract_values()
U_arred = modelo_arred.U.extract_values()

# print(Arred, '\n')
# print(Sigm, '\n')
# print(U_sig, '\n')
# print(X_sig, '\n')

# for i in modelo_arred.Pg.extract_values():
#     Arred += modelo_arred.Pg[i].value
#     print(i)



In [188]:

# Arred é um dict com chaves sendo um par (UTE, hora) e valores sendo o valor da PG. converter para dataframe com as colunas sendo as UTEs e as linhas sendo as horas
# criar df com as colunas sendo as UTEs e as linhas sendo as horas:
df_arred = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
for i in Arred:
    df_arred[i[0]][i[1]] = Arred[i]

df_arred

Nome,UTE1,UTE2,UTE3,UTE4
Hora,,,,
Hora1,436.813187,150.0,113.128207,109.955557
Hora2,455.0,266.601549,31.53813,30.289127
Hora3,455.0,321.829263,31.038305,29.892378
Hora4,455.0,278.237399,31.428966,30.202848
Hora5,413.636368,445.681441,113.888693,111.062531
Hora6,413.636368,445.681441,113.888693,111.062531
Hora7,413.636368,445.681441,113.888693,111.062531
Hora8,455.0,215.526293,32.044072,30.686288


In [189]:

# Sigm é um dict com chaves sendo um par (UTE, hora) e valores sendo o valor da PG. converter para dataframe com as colunas sendo as UTEs e as linhas sendo as horas
# criar df com as colunas sendo as UTEs e as linhas sendo as horas:
df_sigm = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
for i in Sigm:
    df_sigm[i[0]][i[1]] = Sigm[i]

df_sigm

Nome,UTE1,UTE2,UTE3,UTE4
Hora,,,,
Hora1,436.81319,150.0,123.63129,122.504507
Hora2,455.0,266.601555,24.339094,23.852837
Hora3,455.0,321.829266,24.147771,23.701786
Hora4,455.0,278.237404,24.29731,23.819988
Hora5,413.636368,451.583643,123.91499,122.90142
Hora6,413.636368,451.583643,123.91499,122.90142
Hora7,413.636368,451.583643,123.91499,122.90142
Hora8,455.0,215.526303,24.532713,24.004045


In [190]:

# U_sig é um dict com chaves sendo um par (UTE, hora) e valores sendo o valor da U. converter para dataframe com as colunas sendo as UTEs e as linhas sendo as horas
# criar df com as colunas sendo as UTEs e as linhas sendo as horas:
df_U_sig = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
for i in U_sig:
    df_U_sig[i[0]][i[1]] = U_sig[i]

df_U_sig

Nome,UTE1,UTE2,UTE3,UTE4
Hora,,,,
Hora1,1.0,0.087912,0.0,0.0
Hora2,1.0,0.281319,0.0,0.0
Hora3,1.0,0.450549,0.0,0.0
Hora4,1.0,0.305494,0.0,0.0
Hora5,0.967033,0.0,0.0,0.0
Hora6,0.676923,0.0,0.0,0.0
Hora7,0.701099,0.0,0.0,0.0
Hora8,1.0,0.208791,0.0,0.0


In [191]:

# U_arred é um dict com chaves sendo um par (UTE, hora) e valores sendo o valor da U. converter para dataframe com as colunas sendo as UTEs e as linhas sendo as horas
# criar df com as colunas sendo as UTEs e as linhas sendo as horas:
df_U_arred = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
for i in U_arred:
    df_U_arred[i[0]][i[1]] = U_arred[i]

df_U_arred

Nome,UTE1,UTE2,UTE3,UTE4
Hora,,,,
Hora1,1.0,0.087912,0.0,0.0
Hora2,1.0,0.281319,0.0,0.0
Hora3,1.0,0.450549,0.0,0.0
Hora4,1.0,0.305494,0.0,0.0
Hora5,0.967033,0.0,0.0,0.0
Hora6,0.676923,0.0,0.0,0.0
Hora7,0.701099,0.0,0.0,0.0
Hora8,1.0,0.208791,0.0,0.0


In [192]:
Dem 

,Hora,Dem,Res
0,Hora1,450,45
1,Hora2,530,53
2,Hora3,600,60
3,Hora4,540,54
4,Hora5,400,40
5,Hora6,280,28
6,Hora7,290,29
7,Hora8,500,50


In [193]:
UTE

,Nome,a,b,c,PGmax,PGmin
0,UTE1,1000.0,16.19,0.00048,455.0,150.0
1,UTE2,970.0,17.26,0.00031,455.0,150.0
2,UTE3,700.0,16.60,0.00200,130.0,20.0
3,UTE4,680.0,16.50,0.00211,130.0,20.0


In [197]:
# ordenar, para cada hora, as UTEs em ordem decrescente de U
ordem_despacho = np.zeros((len(Dem['Hora']), len(UTE['Nome'])))
for i in range(len(Dem['Hora'])):
    # guardar o número da UTE 
    ordem_despacho[i] = np.argsort(df_U_sig.iloc[i])[::-1]

ordem_despacho


array([[0., 1., 3., 2.],
       [0., 1., 3., 2.],
       [0., 1., 3., 2.],
       [0., 1., 3., 2.],
       [0., 3., 2., 1.],
       [0., 3., 2., 1.],
       [0., 3., 2., 1.],
       [0., 1., 3., 2.]])